# Análise

In [180]:
trainProcessed = pd.read_csv("./processedData/trainProcessed.csv", )

In [181]:
trainProcessed[trainProcessed['Phrase'].notnull()]

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,seri escapad demonstr adag good goos good gand...,1
1,64,2,quiet introspect entertain independ worth seek,4
2,82,3,fan ismail merchant work suspect hard time sit,1
3,117,4,posit thrill combin ethnographi intrigu betray...,3
4,157,5,aggress self glorif manipul whitewash,1
...,...,...,...,...
8524,155985,8540,will claustrophob concept,2
8525,155998,8541,despit annoy capabl clayburgh tambor great job...,2
8526,156022,8542,tri parodi genr joke unit state,1
8527,156032,8543,movi downfal substitut plot person,1


In [59]:
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import Normalizer
from sklearn import metrics

In [153]:
aux = trainProcessed[trainProcessed['Phrase'].notnull()]

In [154]:
aux.head

<bound method NDFrame.head of       PhraseId  SentenceId                                             Phrase  \
0            1           1  seri escapad demonstr adag good goos good gand...   
1           64           2     quiet introspect entertain independ worth seek   
2           82           3     fan ismail merchant work suspect hard time sit   
3          117           4  posit thrill combin ethnographi intrigu betray...   
4          157           5              aggress self glorif manipul whitewash   
...        ...         ...                                                ...   
8524    155985        8540                          will claustrophob concept   
8525    155998        8541  despit annoy capabl clayburgh tambor great job...   
8526    156022        8542                    tri parodi genr joke unit state   
8527    156032        8543                 movi downfal substitut plot person   
8528    156040        8544  film darkli atmospher herrmann quietli suggest...  

In [155]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor

In [156]:
x = aux.loc[:, ["Phrase"]]
y = aux.loc[:, ["Sentiment"]]

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state = 27, train_size = 0.8)

In [165]:
x_train = list(x_train["Phrase"])
print(type(x_train), "\n", len(x_train), "\n", type(x_train[0]))

x_test = list(x_test["Phrase"])
print(type(x_test), "\n", len(x_test), "\n", type(x_test[0]))

<class 'list'> 
 6797 
 <class 'str'>


In [166]:
vectorizer=CountVectorizer(min_df=1)

dtmTF_train = vectorizer.fit_transform(x_train)
df_tf_train = pd.DataFrame(dtmTF_train.toarray(), index=range(0, dtmTF_train.shape[0]), columns=vectorizer.get_feature_names())
df_tf_train.head(10)

dtmTF_test = vectorizer.fit_transform(x_test)
df_tf_test = pd.DataFrame(dtmTF_test.toarray(), index=range(0, dtmTF_test.shape[0]), columns=vectorizer.get_feature_names())
df_tf_test.head(10)

In [169]:
tfidf_transformer = TfidfTransformer()
dtmTF_IDF_train = tfidf_transformer.fit_transform(dtmTF_train)

df_tf_idf_train = pd.DataFrame(dtmTF_IDF_train.toarray(), index=range(0, dtmTF_IDF_train.shape[0]), columns=vectorizer.get_feature_names())
df_tf_idf_train.head(10)

<6797x9214 sparse matrix of type '<class 'numpy.float64'>'
	with 56582 stored elements in Compressed Sparse Row format>

In [ ]:
tfidf_transformer = TfidfTransformer()
dtmTF_IDF_test = tfidf_transformer.fit_transform(dtmTF_test)

df_tf_idf_test = pd.DataFrame(dtmTF_IDF_test.toarray(), index=range(0, dtmTF_IDF_test.shape[0]), columns=vectorizer.get_feature_names())
df_tf_idf_test.head(10)

In [171]:
lsa = TruncatedSVD(25, algorithm='randomized')
dtm_lsa_tf_train = lsa.fit_transform(dtmTF_train)
dtm_lsa_tf_train = Normalizer(copy=False).fit_transform(dtm_lsa_tf_train)

dtm_lsa_tf_idf_train = lsa.fit_transform(dtmTF_IDF_train)
dtm_lsa_tf_idf_train = Normalizer(copy=False).fit_transform(dtm_lsa_tf_idf_train)

In [ ]:
lsa = TruncatedSVD(25, algorithm='randomized')
dtm_lsa_tf_test = lsa.fit_transform(dtmTF_test)
dtm_lsa_tf_test = Normalizer(copy=False).fit_transform(dtm_lsa_tf_test)

dtm_lsa_tf_idf_test = lsa.fit_transform(dtmTF_IDF_test)
dtm_lsa_tf_idf_test = Normalizer(copy=False).fit_transform(dtm_lsa_tf_idf_test)

In [172]:
print(dtm_lsa_tf_train.shape[0])
print(dtm_lsa_tf_idf_train.shape)

print(dtm_lsa_tf_test.shape[0])
print(dtm_lsa_tf_idf_test.shape)

6797
(6797, 25)


In [173]:
df_lsa_tf_train = pd.DataFrame(dtm_lsa_tf_train, index=range(0, dtm_lsa_tf_train.shape[0]))
df_lsa_tf_train.head(10)

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,0.403043,0.022870,0.053479,0.344231,-0.087819,0.158178,-0.161666,0.395887,-0.121862,0.007871,...,0.147136,0.277966,0.210963,-0.206989,-0.148562,-0.041655,-0.121851,0.129230,-0.087253,-0.136891
1,0.377101,-0.505138,-0.129687,-0.055187,-0.072034,-0.044365,-0.001141,0.072722,0.004313,-0.052363,...,0.184976,0.094777,-0.176262,-0.308334,-0.497129,-0.035795,0.121741,-0.031653,0.047602,0.244803
2,0.094139,-0.023511,0.081116,0.118826,-0.119323,0.069488,-0.075302,0.089659,0.804431,0.033744,...,0.037873,-0.073967,-0.005969,0.017766,-0.055868,-0.006735,-0.001204,-0.035344,0.045233,0.018746
3,0.115805,-0.015486,0.088388,0.097512,-0.113996,0.053163,-0.042607,0.194611,0.046485,0.049647,...,0.477383,-0.347421,-0.252148,-0.267533,-0.448127,0.041382,0.064150,-0.055985,0.085919,0.234095
4,0.218291,-0.081384,0.854268,-0.312855,0.163951,-0.080380,-0.003272,-0.004793,-0.095412,0.059640,...,-0.018279,-0.010864,0.030128,-0.003005,-0.084226,-0.116521,0.001302,0.027901,-0.024412,-0.003349
5,0.124962,-0.036658,0.106687,0.235398,-0.112398,0.745514,0.538715,-0.128713,-0.105911,0.036541,...,-0.004858,0.009497,-0.064227,0.077408,-0.041284,-0.024648,0.050460,-0.037605,0.007420,-0.055782
6,0.107905,-0.040546,0.242350,0.045072,0.043227,0.056116,-0.002781,-0.134161,0.270991,-0.427846,...,0.054432,-0.018269,0.047096,-0.020312,0.093867,0.021323,-0.036955,0.042233,0.014605,-0.014589
7,0.213645,0.098951,0.113127,0.196795,-0.162533,0.218391,-0.104470,0.161945,0.152869,0.072816,...,-0.064880,0.096812,0.052023,-0.271742,-0.189888,-0.096726,-0.357539,-0.036331,0.133766,-0.214130
8,0.092492,-0.022661,0.064938,0.100369,-0.031023,0.076655,-0.056300,0.186518,0.005693,0.103393,...,0.448582,-0.648907,-0.193767,0.002162,-0.075379,0.111395,-0.107026,-0.077587,0.043182,-0.037117
9,0.196395,0.042851,0.094373,0.190006,-0.401215,-0.088153,-0.247985,-0.075817,0.149061,-0.007339,...,-0.008184,-0.064922,0.008440,0.182034,0.063074,-0.046841,-0.009867,0.397953,-0.161153,-0.581798


In [ ]:
df_lsa_tf_test = pd.DataFrame(dtm_lsa_tf_test, index=range(0, dtm_lsa_tf_test.shape[0]))
df_lsa_tf_test.head(10)

In [174]:
df_lsa_tf_idf_train = pd.DataFrame(dtm_lsa_tf_idf_train, index=range(0, dtm_lsa_tf_idf_train.shape[0]))
df_lsa_tf_idf_train.head(10)

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,0.429345,-0.232455,-0.000359,0.028924,0.072200,0.006151,0.262607,-0.199899,0.028455,0.320294,...,-0.261215,-0.128806,0.144967,0.169382,0.121002,0.200028,-0.185936,-0.027604,0.118641,-0.382399
1,0.389206,0.286037,-0.096905,-0.029026,-0.088621,-0.124818,0.017487,-0.121495,-0.106671,0.032867,...,-0.074582,-0.422571,0.509370,-0.022132,-0.332603,-0.219315,0.072673,0.033523,0.105573,-0.164157
2,0.166976,-0.032655,-0.023663,0.122479,0.067591,-0.174236,0.305702,0.881973,-0.047975,-0.007792,...,-0.018299,-0.011103,0.005850,-0.055412,0.024147,-0.077173,0.040630,0.016104,0.049165,-0.079698
3,0.187316,-0.075576,-0.042341,0.122370,0.079286,-0.011455,0.090386,-0.108745,-0.037433,0.016573,...,-0.268091,-0.438429,0.492580,-0.059029,0.276173,-0.319233,0.301158,0.074460,0.016698,-0.106560
4,0.320701,-0.062341,-0.127004,0.659704,0.000925,0.388488,-0.344448,0.026536,-0.119746,0.017186,...,0.074801,-0.053916,0.133949,-0.083932,0.042956,-0.049586,-0.004941,-0.100509,-0.077832,0.050262
5,0.214469,-0.029597,0.108346,0.179935,0.157724,-0.128424,0.199733,-0.135254,0.736038,-0.074715,...,0.151047,0.008236,0.044288,0.055877,-0.099476,-0.085145,-0.062294,-0.081141,-0.185250,-0.085637
6,0.195699,-0.023624,0.025919,0.291958,0.068321,0.075574,-0.144664,0.018069,0.011535,-0.069192,...,-0.145372,0.034753,0.176128,0.188227,0.285502,0.091769,-0.143680,-0.006315,-0.157121,-0.109138
7,0.343843,-0.333799,-0.049489,0.195323,0.164894,-0.043868,0.222108,-0.213281,0.070126,0.286247,...,-0.056523,0.024443,0.099619,0.309945,-0.055675,0.024884,-0.083916,-0.346220,0.336473,0.017853
8,0.138981,-0.033454,-0.004165,0.050833,0.108971,0.025605,0.047173,-0.072071,0.022821,0.054319,...,-0.176987,-0.201983,0.124124,0.022033,0.674775,-0.268500,0.200156,-0.118853,-0.110034,0.361700
9,0.257952,-0.189155,-0.157326,0.219186,0.060134,-0.221628,0.349377,0.111426,-0.184287,0.043425,...,0.237318,0.029240,-0.339096,-0.342512,-0.015455,0.046336,0.136289,-0.451059,-0.169902,0.070751


In [ ]:
df_lsa_tf_idf_test = pd.DataFrame(dtm_lsa_tf_idf_test, index=range(0, dtm_lsa_tf_idf_test.shape[0]))
df_lsa_tf_idf_test.head(10)

In [175]:
from sklearn.tree import DecisionTreeRegressor

In [179]:
regTF_train = DecisionTreeRegressor(max_depth = 10, random_state = 27)
regTF_IDF_train = DecisionTreeRegressor(max_depth = 10, random_state = 27)

regTF_train.fit(dtm_lsa_tf_train, y_train)
regTF_IDF_train.fit(dtm_lsa_tf_idf_train, y_train)

DecisionTreeRegressor(max_depth=10, random_state=27)

In [ ]:
regTF_test = DecisionTreeRegressor(max_depth = 10, random_state = 27)
regTF_IDF_test = DecisionTreeRegressor(max_depth = 10, random_state = 27)

regTF_test.fit(dtm_lsa_tf_test, y_test)
regTF_IDF_test.fit(dtm_lsa_tf_idf_test, y_test)

In [ ]:
regTF.predict()